In [1]:
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern
import numpy as np
import os
import cv2
import time
import pandas as pd

In [2]:
def load_images():
    dir = os.path.join(os.sep,'home','mohit','Desktop','dataset')
    images = {}
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    images['c'+str(k)+'_'+i+'_'+j] = cv2.imread(os.path.join(dir,'Cheque'+str(k),i,i+'_'+j+'.tif'))
                    images['c'+str(k)+'_'+i+'_'+j] = cv2.cvtColor(images['c'+str(k)+'_'+i+'_'+j], cv2.COLOR_BGR2YCrCb)
                except:
                    break
    return images

In [3]:
def n_size(images):
    size = {} #no. of ink pixels .. considering all non white pixels(<th) as ink pixels
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                count = 0
                try:
                    image = images['c'+str(k)+'_'+i+'_'+j][:,:,0]
                    for a in range(0,image.shape[0]):
                        for b in range(0,image.shape[1]):
                            if image[a][b] != 255:
                                count+=1
                    size['c'+str(k)+'_'+i+'_'+j] = count
                   # print('c'+str(k)+'_'+i+'_'+j,count)
                except:
                    #print('..')
                    break  
    return size

In [4]:
def mask(image):
    r = (image==255)
    return r


In [5]:
def get_pixel_else_0(l, idx, idy, default=0):
    try:
        return l[idx,idy]
    except IndexError:
        return default

    
def find_variations(pixel_values):
    prev = pixel_values[-1]
    t = 0
    for p in range(0, len(pixel_values)):
        cur = pixel_values[p]
        if cur != prev:
            t += 1
        prev = cur
    return t    


In [6]:
def si(x,threshold):
    if x>threshold:
        return 1
    else:
        return 0

def OCS_LBP(img,encoding='riu2'):
    ocslbp = np.zeros_like(img,dtype=np.float64)

    for x in range(0, len(ocslbp)):
        for y in range(0, len(ocslbp[0])):
            center = img[x,y]
            top_left      = get_pixel_else_0(img, x-1, y-1)
            top_up        = get_pixel_else_0(img, x, y-1)
            top_right     = get_pixel_else_0(img, x+1, y-1)
            right         = get_pixel_else_0(img, x+1, y )
            left          = get_pixel_else_0(img, x-1, y )
            bottom_left   = get_pixel_else_0(img, x-1, y+1)
            bottom_right  = get_pixel_else_0(img, x+1, y+1)
            bottom_down   = get_pixel_else_0(img, x,   y+1 )

            v1 = si((int(right)+int(left))/2,center)
            v2 = si((int(bottom_right)+int(top_left))/2,center)
            v3 = si((int(bottom_down)+int(top_up))/2,center)
            v4 = si((int(bottom_left)+int(top_right))/2,center)
            
            va = [v1,v2,v3,v4]
            
            if(encoding=='riu2'):
                if(find_variations(va)<=2):
                    ocslbp.itemset((x,y), va.count(0))
                else :
                    ocslbp.itemset((x,y),5)
            else:
                weights = [1, 2, 4, 8]
                res = 0
                for a in range(0, len(weights)):
                    res += weights[a] * va[a]
                ocslbp.itemset((x,y),res)
                

    return ocslbp

In [7]:
def OCS_CLBP_M(img,c,encoding='riu2'):
    ocsclbpm = np.zeros_like(img,dtype=np.float64)

    for x in range(0, len(ocsclbpm)):
        for y in range(0, len(ocsclbpm[0])):
            center = img[x,y]
            top_left      = get_pixel_else_0(img, x-1, y-1)
            top_up        = get_pixel_else_0(img, x, y-1)
            top_right     = get_pixel_else_0(img, x+1, y-1)
            right         = get_pixel_else_0(img, x+1, y )
            left          = get_pixel_else_0(img, x-1, y )
            bottom_left   = get_pixel_else_0(img, x-1, y+1)
            bottom_right  = get_pixel_else_0(img, x+1, y+1)
            bottom_down   = get_pixel_else_0(img, x,   y+1 )
            
            mp1 = abs((int(right)+int(left))/2-center)
            mp2 = abs((int(bottom_right)+int(top_left))/2-center)
            mp3 = abs((int(bottom_down)+int(top_up))/2-center)
            mp4 = abs((int(bottom_left)+int(top_right))/2-center)
            
            t = lambda x,c: (x>=c)*1
            
            v1 = t(mp1,c)
            v2 = t(mp2,c)
            v3 = t(mp3,c)
            v4 = t(mp4,c)

            va = [v1,v2,v3,v4]
        
            if(encoding=='riu2'):
                if(find_variations(va)<=2):
                    ocsclbpm.itemset((x,y), va.count(0))
                else :
                    ocsclbpm.itemset((x,y),5)
            else:
                weights = [1, 2, 4, 8]
                res = 0
                for a in range(0, len(weights)):
                    res += weights[a] * va[a]
                ocsclbpm.itemset((x,y),res)
            
    return ocsclbpm

In [8]:
def LBP_CI(image,beta):
    return (image>beta)*1

def calc_c(img,img1,size):
    c  =  0
    beta = 0
    for x in range(0, len(img)):
        for y in range(0, len(img[0])):
            if(img1[x,y]!=255):
                beta+=img[x,y]
                center = img[x,y]
                top_left      = get_pixel_else_0(img, x-1, y-1)
                top_up        = get_pixel_else_0(img, x, y-1)
                top_right     = get_pixel_else_0(img, x+1, y-1)
                right         = get_pixel_else_0(img, x+1, y )
                left          = get_pixel_else_0(img, x-1, y )
                bottom_left   = get_pixel_else_0(img, x-1, y+1)
                bottom_right  = get_pixel_else_0(img, x+1, y+1)
                bottom_down   = get_pixel_else_0(img, x,   y+1 )
                mp1 = abs((int(right)+int(left))/2-center)
                mp2 = abs((int(bottom_right)+int(top_left))/2-center)
                mp3 = abs((int(bottom_down)+int(top_up))/2-center)
                mp4 = abs((int(bottom_left)+int(top_right))/2-center)
                
                c+= np.mean([mp1,mp2,mp3,mp4])
    
    c/=size
    beta/=size
    return c,beta

def OCS_CLBP(images,dim,sizes,encoding='riu2',alpha=1):
    hist_S = {}
    hist_M = {}
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    print(k,i,j)

                    s = sizes['c'+str(k)+'_'+i+'_'+j]
                    c,beta = calc_c(images['c'+str(k)+'_'+i+'_'+j][:,:,dim],images['c'+str(k)+'_'+i+'_'+j][:,:,0],s)
                    c*=alpha

                    lbp_pens_s = OCS_LBP(images['c'+str(k)+'_'+i+'_'+j][:,:,dim],encoding)
                    lbp_pens_m = OCS_CLBP_M(images['c'+str(k)+'_'+i+'_'+j][:,:,dim],c,encoding)

                    masc = LBP_CI(images['c'+str(k)+'_'+i+'_'+j][:,:,dim],beta)

                    if(encoding=='riu2'):
                        lbp_pens_s+=6*masc
                        lbp_pens_m+=6*masc
                        r=13
                    else:
                        lbp_pens_s+=16*masc
                        lbp_pens_m+=16*masc
                        r=33

                    lbp_pens_s+= (mask(images['c'+str(k)+'_'+i+'_'+j][:,:,0])*100)
                    lbp_pens_m+= (mask(images['c'+str(k)+'_'+i+'_'+j][:,:,0])*100)

                    hist_s,_ = np.histogram(lbp_pens_s.ravel(),range(0,r))
                    hist_m,_ = np.histogram(lbp_pens_m.ravel(),range(0,r))

                    hist_S['c'+str(k)+'_'+i+'_'+j] = hist_s/s
                    hist_M['c'+str(k)+'_'+i+'_'+j] = hist_m/s

                except:
                    break
                   
    return hist_S,hist_M

In [9]:
def dist(param1,param2,n1,n2):
    mean1 = np.mean(param1)
    mean2 = np.mean(param2)
    var1  = np.var(param1)
    var2 = np.var(param2)
     
    d = np.abs(mean1-mean2)
    d/=(var1/n1 + var2/n2)**0.5
    
    return d

def compute_dist(param1,param2,n1,n2):
    d = 0
    for i in range(0,len(param1)):
        d+=np.nan_to_num(((param1[i]-param2[i])**2)/(param1[i]+param2[i]))
    return d

def distance_distribution(features,sizes):
    same_pens = {}
    diff_pens = {}
    
    for k in range(1,113) :
        for i in ['1','2']: 
            for n in ['1','2']:
                for j in ['1','2','3','4','5','6','7','8'] :
                    for m in ['1','2','3','4','5','6','7','8'] :
                        try:
                            if(i==n and j==m):
                                continue    
                            elif i==n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in same_pens:
                                same_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = np.nan_to_num(compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m],sizes['c'+str(k)+'_'+i+'_'+j],sizes['c'+str(k)+'_'+n+'_'+m]))
                                #print(' same {}   :{}'.format('c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m,same_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m]))
                    
                            elif i!=n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in diff_pens :                               
                                diff_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = np.nan_to_num(compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m],sizes['c'+str(k)+'_'+i+'_'+j],sizes['c'+str(k)+'_'+n+'_'+m]))
                                #print(' diff {}   :{}'.format('c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m,diff_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m]))
                        except:
                            
                            pass
    print('Lengths same pens-{} different pens-{}'.format(len(same_pens),len(diff_pens)))
    return same_pens,diff_pens

In [10]:
images = load_images()

In [11]:
sizes = n_size(images)

# using riu2 encoding 

In [28]:
histY_s,histY_m = OCS_CLBP(images,0,sizes)

1 1 1
1 1 2
1 1 3
1 2 1
1 2 2
1 2 3
1 2 4
1 2 5
1 2 6
2 1 1
2 1 2
2 1 3
2 1 4
2 1 5
2 1 6
2 1 7
2 2 1
2 2 2
2 2 3
3 1 1
3 1 2
3 1 3
3 1 4
3 1 5
3 1 6
3 2 1
3 2 2
3 2 3
3 2 4
4 1 1
4 1 2
4 1 3
4 1 4
4 1 5
4 1 6
4 1 7
4 1 8
4 2 1
4 2 2
4 2 3
5 1 1
5 1 2
5 1 3
5 1 4
5 1 5
5 1 6
5 1 7
5 2 1
5 2 2
5 2 3
6 1 1
6 1 2
6 1 3
6 1 4
6 1 5
6 1 6
6 1 7
6 2 1
6 2 2
6 2 3
7 1 1
7 1 2
7 1 3
7 1 4
7 1 5
7 1 6
7 2 1
7 2 2
7 2 3
8 1 1
8 1 2
8 1 3
8 1 4
8 1 5
8 1 6
8 1 7
8 1 8
8 2 1
8 2 2
8 2 3
9 1 1
9 1 2
9 1 3
9 1 4
9 1 5
9 1 6
9 1 7
9 1 8
9 2 1
9 2 2
9 2 3
10 1 1
10 1 2
10 1 3
10 1 4
10 1 5
10 2 1
10 2 2
10 2 3
10 2 4
11 1 1
11 1 2
11 1 3
11 1 4
11 1 5
11 1 6
11 1 7
11 1 8
11 2 1
11 2 2
11 2 3
12 1 1
12 1 2
12 1 3
12 1 4
12 1 5
12 1 6
12 1 7
12 2 1
12 2 2
12 2 3
13 1 1
13 1 2
13 1 3
13 1 4
13 1 5
13 2 1
13 2 2
13 2 3
13 2 4
14 1 1
14 1 2
14 1 3
14 1 4
14 1 5
14 1 6
14 1 7
14 2 1
14 2 2
14 2 3
15 1 1
15 1 2
15 1 3
15 1 4
15 1 5
15 2 1
15 2 2
15 2 3
16 1 1
16 1 2
16 1 3
16 1 4
16 1 5
16 1 6
16 1 7
16 2 1

In [31]:
histCb_s,histCb_m = OCS_CLBP(images,1,sizes)
histCr_s,histCr_m = OCS_CLBP(images,2,sizes)

1 1 1
1 1 2
1 1 3
1 2 1
1 2 2
1 2 3
1 2 4
1 2 5
1 2 6
2 1 1
2 1 2
2 1 3
2 1 4
2 1 5
2 1 6
2 1 7
2 2 1
2 2 2
2 2 3
3 1 1
3 1 2
3 1 3
3 1 4
3 1 5
3 1 6
3 2 1
3 2 2
3 2 3
3 2 4
4 1 1
4 1 2
4 1 3
4 1 4
4 1 5
4 1 6
4 1 7
4 1 8
4 2 1
4 2 2
4 2 3
5 1 1
5 1 2
5 1 3
5 1 4
5 1 5
5 1 6
5 1 7
5 2 1
5 2 2
5 2 3
6 1 1
6 1 2
6 1 3
6 1 4
6 1 5
6 1 6
6 1 7
6 2 1
6 2 2
6 2 3
7 1 1
7 1 2
7 1 3
7 1 4
7 1 5
7 1 6
7 2 1
7 2 2
7 2 3
8 1 1
8 1 2
8 1 3
8 1 4
8 1 5
8 1 6
8 1 7
8 1 8
8 2 1
8 2 2
8 2 3
9 1 1
9 1 2
9 1 3
9 1 4
9 1 5
9 1 6
9 1 7
9 1 8
9 2 1
9 2 2
9 2 3
10 1 1
10 1 2
10 1 3
10 1 4
10 1 5
10 2 1
10 2 2
10 2 3
10 2 4
11 1 1
11 1 2
11 1 3
11 1 4
11 1 5
11 1 6
11 1 7
11 1 8
11 2 1
11 2 2
11 2 3
12 1 1
12 1 2
12 1 3
12 1 4
12 1 5
12 1 6
12 1 7
12 2 1
12 2 2
12 2 3
13 1 1
13 1 2
13 1 3
13 1 4
13 1 5
13 2 1
13 2 2
13 2 3
13 2 4
14 1 1
14 1 2
14 1 3
14 1 4
14 1 5
14 1 6
14 1 7
14 2 1
14 2 2
14 2 3
15 1 1
15 1 2
15 1 3
15 1 4
15 1 5
15 2 1
15 2 2
15 2 3
16 1 1
16 1 2
16 1 3
16 1 4
16 1 5
16 1 6
16 1 7
16 2 1

11 1 5
11 1 6
11 1 7
11 1 8
11 2 1
11 2 2
11 2 3
12 1 1
12 1 2
12 1 3
12 1 4
12 1 5
12 1 6
12 1 7
12 2 1
12 2 2
12 2 3
13 1 1
13 1 2
13 1 3
13 1 4
13 1 5
13 2 1
13 2 2
13 2 3
13 2 4
14 1 1
14 1 2
14 1 3
14 1 4
14 1 5
14 1 6
14 1 7
14 2 1
14 2 2
14 2 3
15 1 1
15 1 2
15 1 3
15 1 4
15 1 5
15 2 1
15 2 2
15 2 3
16 1 1
16 1 2
16 1 3
16 1 4
16 1 5
16 1 6
16 1 7
16 2 1
16 2 2
16 2 3
17 1 1
17 1 2
17 1 3
17 1 4
17 1 5
17 1 6
17 2 1
17 2 2
17 2 3
18 1 1
18 1 2
18 1 3
18 1 4
18 1 5
18 1 6
18 1 7
18 2 1
18 2 2
18 2 3
18 2 4
19 1 1
19 1 2
19 1 3
19 1 4
19 1 5
19 1 6
19 2 1
19 2 2
19 2 3
20 1 1
20 1 2
20 1 3
20 1 4
20 1 5
20 1 6
20 1 7
20 2 1
20 2 2
20 2 3
21 1 1
21 1 2
21 1 3
21 1 4
21 2 1
21 2 2
21 2 3
22 1 1
22 1 2
22 1 3
22 1 4
22 1 5
22 1 6
22 1 7
22 2 1
22 2 2
22 2 3
22 2 4
22 2 5
23 1 1
23 1 2
23 1 3
23 1 4
23 1 5
23 1 6
23 1 7
23 2 1
23 2 2
23 2 3
24 1 1
24 1 2
24 1 3
24 1 4
24 1 5
24 2 1
24 2 2
24 2 3
25 1 1
25 1 2
25 1 3
25 1 4
25 1 5
25 1 6
25 1 7
25 1 8
25 2 1
25 2 2
25 2 3
25 2 4
26 1 1

In [32]:
same_pensY_s,diff_pensY_s = distance_distribution(histY_s,sizes)
same_pensY_s = [ v for v in same_pensY_s. values() ]
diff_pensY_s = [ v for v in diff_pensY_s. values() ]

same_pensY_m,diff_pensY_m = distance_distribution(histY_m,sizes)
same_pensY_m = [ v for v in same_pensY_m. values() ]
diff_pensY_m = [ v for v in diff_pensY_m. values() ]

dy_s=dist(same_pensY_s,diff_pensY_s,1465,1456)
dy_m=dist(same_pensY_m,diff_pensY_m,1465,1456)
dy_s,dy_m

/home/mohit/.local/lib/python3.5/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456


(29.978091743832408, 24.34360888028348)

In [33]:
same_pensCb_s,diff_pensCb_s = distance_distribution(histCb_s,sizes)
same_pensCb_s = [ v for v in same_pensCb_s. values() ]
diff_pensCb_s = [ v for v in diff_pensCb_s. values() ]

same_pensCb_m,diff_pensCb_m = distance_distribution(histCb_m,sizes)
same_pensCb_m = [ v for v in same_pensCb_m. values() ]
diff_pensCb_m = [ v for v in diff_pensCb_m. values() ]

dcb_s=dist(same_pensCb_s,diff_pensCb_s,1465,1456)
dcb_m=dist(same_pensCb_m,diff_pensCb_m,1465,1456)

/home/mohit/.local/lib/python3.5/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456


In [34]:
same_pensCr_s,diff_pensCr_s = distance_distribution(histCr_s,sizes)
same_pensCr_s = [ v for v in same_pensCr_s. values() ]
diff_pensCr_s = [ v for v in diff_pensCr_s. values() ]

same_pensCr_m,diff_pensCr_m = distance_distribution(histCr_m,sizes)
same_pensCr_m = [ v for v in same_pensCr_m. values() ]
diff_pensCr_m = [ v for v in diff_pensCr_m. values() ]

dcr_s=dist(same_pensCr_s,diff_pensCr_s,1465,1456)
dcr_m=dist(same_pensCr_m,diff_pensCr_m,1465,1456)

Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456


In [35]:
print(dy_s,dcb_s,dcr_s)
print(dy_m,dcb_m,dcr_m)

29.978091743832408 25.19955987671866 19.151172339659496
24.34360888028348 12.777687485238548 17.900891448940815


In [36]:
Y = pd.DataFrame(histY_s)
Cb = pd.DataFrame(histCb_s)
Cr = pd.DataFrame(histCr_s)

all_features = pd.concat([Y,Cb,Cr])
all_features.to_csv('/home/mohit/Desktop/featuresUpdated/OCS-CLBP_S|c_riu2.csv',sep = ',')

In [37]:
Y = pd.DataFrame(histY_m)
Cb = pd.DataFrame(histCb_m)
Cr = pd.DataFrame(histCr_m)

all_features = pd.concat([Y,Cb,Cr])
all_features.to_csv('/home/mohit/Desktop/featuresUpdated/OCS-CLBP_M|c_riu2.csv',sep = ',')

# using default encoding

In [12]:
histY_s,histY_m = OCS_CLBP(images,0,sizes,encoding='default')
histCb_s,histCb_m = OCS_CLBP(images,1,sizes,encoding='default')
histCr_s,histCr_m = OCS_CLBP(images,2,sizes,encoding='default')

1 1 1
1 1 2
1 1 3
1 2 1
1 2 2
1 2 3
1 2 4
1 2 5
1 2 6
2 1 1
2 1 2
2 1 3
2 1 4
2 1 5
2 1 6
2 1 7
2 2 1
2 2 2
2 2 3
3 1 1
3 1 2
3 1 3
3 1 4
3 1 5
3 1 6
3 2 1
3 2 2
3 2 3
3 2 4
4 1 1
4 1 2
4 1 3
4 1 4
4 1 5
4 1 6
4 1 7
4 1 8
4 2 1
4 2 2
4 2 3
5 1 1
5 1 2
5 1 3
5 1 4
5 1 5
5 1 6
5 1 7
5 2 1
5 2 2
5 2 3
6 1 1
6 1 2
6 1 3
6 1 4
6 1 5
6 1 6
6 1 7
6 2 1
6 2 2
6 2 3
7 1 1
7 1 2
7 1 3
7 1 4
7 1 5
7 1 6
7 2 1
7 2 2
7 2 3
8 1 1
8 1 2
8 1 3
8 1 4
8 1 5
8 1 6
8 1 7
8 1 8
8 2 1
8 2 2
8 2 3
9 1 1
9 1 2
9 1 3
9 1 4
9 1 5
9 1 6
9 1 7
9 1 8
9 2 1
9 2 2
9 2 3
10 1 1
10 1 2
10 1 3
10 1 4
10 1 5
10 2 1
10 2 2
10 2 3
10 2 4
11 1 1
11 1 2
11 1 3
11 1 4
11 1 5
11 1 6
11 1 7
11 1 8
11 2 1
11 2 2
11 2 3
12 1 1
12 1 2
12 1 3
12 1 4
12 1 5
12 1 6
12 1 7
12 2 1
12 2 2
12 2 3
13 1 1
13 1 2
13 1 3
13 1 4
13 1 5
13 2 1
13 2 2
13 2 3
13 2 4
14 1 1
14 1 2
14 1 3
14 1 4
14 1 5
14 1 6
14 1 7
14 2 1
14 2 2
14 2 3
15 1 1
15 1 2
15 1 3
15 1 4
15 1 5
15 2 1
15 2 2
15 2 3
16 1 1
16 1 2
16 1 3
16 1 4
16 1 5
16 1 6
16 1 7
16 2 1

11 1 5
11 1 6
11 1 7
11 1 8
11 2 1
11 2 2
11 2 3
12 1 1
12 1 2
12 1 3
12 1 4
12 1 5
12 1 6
12 1 7
12 2 1
12 2 2
12 2 3
13 1 1
13 1 2
13 1 3
13 1 4
13 1 5
13 2 1
13 2 2
13 2 3
13 2 4
14 1 1
14 1 2
14 1 3
14 1 4
14 1 5
14 1 6
14 1 7
14 2 1
14 2 2
14 2 3
15 1 1
15 1 2
15 1 3
15 1 4
15 1 5
15 2 1
15 2 2
15 2 3
16 1 1
16 1 2
16 1 3
16 1 4
16 1 5
16 1 6
16 1 7
16 2 1
16 2 2
16 2 3
17 1 1
17 1 2
17 1 3
17 1 4
17 1 5
17 1 6
17 2 1
17 2 2
17 2 3
18 1 1
18 1 2
18 1 3
18 1 4
18 1 5
18 1 6
18 1 7
18 2 1
18 2 2
18 2 3
18 2 4
19 1 1
19 1 2
19 1 3
19 1 4
19 1 5
19 1 6
19 2 1
19 2 2
19 2 3
20 1 1
20 1 2
20 1 3
20 1 4
20 1 5
20 1 6
20 1 7
20 2 1
20 2 2
20 2 3
21 1 1
21 1 2
21 1 3
21 1 4
21 2 1
21 2 2
21 2 3
22 1 1
22 1 2
22 1 3
22 1 4
22 1 5
22 1 6
22 1 7
22 2 1
22 2 2
22 2 3
22 2 4
22 2 5
23 1 1
23 1 2
23 1 3
23 1 4
23 1 5
23 1 6
23 1 7
23 2 1
23 2 2
23 2 3
24 1 1
24 1 2
24 1 3
24 1 4
24 1 5
24 2 1
24 2 2
24 2 3
25 1 1
25 1 2
25 1 3
25 1 4
25 1 5
25 1 6
25 1 7
25 1 8
25 2 1
25 2 2
25 2 3
25 2 4
26 1 1

20 2 2
20 2 3
21 1 1
21 1 2
21 1 3
21 1 4
21 2 1
21 2 2
21 2 3
22 1 1
22 1 2
22 1 3
22 1 4
22 1 5
22 1 6
22 1 7
22 2 1
22 2 2
22 2 3
22 2 4
22 2 5
23 1 1
23 1 2
23 1 3
23 1 4
23 1 5
23 1 6
23 1 7
23 2 1
23 2 2
23 2 3
24 1 1
24 1 2
24 1 3
24 1 4
24 1 5
24 2 1
24 2 2
24 2 3
25 1 1
25 1 2
25 1 3
25 1 4
25 1 5
25 1 6
25 1 7
25 1 8
25 2 1
25 2 2
25 2 3
25 2 4
26 1 1
26 1 2
26 1 3
26 1 4
26 1 5
26 1 6
26 2 1
26 2 2
26 2 3
26 2 4
27 1 1
27 1 2
27 1 3
27 1 4
27 1 5
27 2 1
27 2 2
27 2 3
27 2 4
27 2 5
28 1 1
28 1 2
28 1 3
28 1 4
28 1 5
28 2 1
28 2 2
28 2 3
28 2 4
29 1 1
29 1 2
29 1 3
29 1 4
29 1 5
29 1 6
29 1 7
29 2 1
29 2 2
29 2 3
30 1 1
30 1 2
30 1 3
30 1 4
30 1 5
30 2 1
30 2 2
30 2 3
30 2 4
31 1 1
31 1 2
31 1 3
31 1 4
31 1 5
31 1 6
31 1 7
31 2 1
31 2 2
31 2 3
32 1 1
32 1 2
32 1 3
32 1 4
32 1 5
32 1 6
32 1 7
32 2 1
32 2 2
32 2 3
32 2 4
33 1 1
33 1 2
33 1 3
33 1 4
33 1 5
33 1 6
33 2 1
33 2 2
33 2 3
33 2 4
34 1 1
34 1 2
34 1 3
34 1 4
34 1 5
34 1 6
34 1 7
34 2 1
34 2 2
34 2 3
35 1 1
35 1 2
35 1 3

In [13]:
same_pensY_s,diff_pensY_s = distance_distribution(histY_s,sizes)
same_pensY_s = [ v for v in same_pensY_s. values() ]
diff_pensY_s = [ v for v in diff_pensY_s. values() ]

same_pensY_m,diff_pensY_m = distance_distribution(histY_m,sizes)
same_pensY_m = [ v for v in same_pensY_m. values() ]
diff_pensY_m = [ v for v in diff_pensY_m. values() ]

dy_s=dist(same_pensY_s,diff_pensY_s,1465,1456)
dy_m=dist(same_pensY_m,diff_pensY_m,1465,1456)

same_pensCb_s,diff_pensCb_s = distance_distribution(histCb_s,sizes)
same_pensCb_s = [ v for v in same_pensCb_s. values() ]
diff_pensCb_s = [ v for v in diff_pensCb_s. values() ]

same_pensCb_m,diff_pensCb_m = distance_distribution(histCb_m,sizes)
same_pensCb_m = [ v for v in same_pensCb_m. values() ]
diff_pensCb_m = [ v for v in diff_pensCb_m. values() ]

dcb_s=dist(same_pensCb_s,diff_pensCb_s,1465,1456)
dcb_m=dist(same_pensCb_m,diff_pensCb_m,1465,1456)

same_pensCr_s,diff_pensCr_s = distance_distribution(histCr_s,sizes)
same_pensCr_s = [ v for v in same_pensCr_s. values() ]
diff_pensCr_s = [ v for v in diff_pensCr_s. values() ]

same_pensCr_m,diff_pensCr_m = distance_distribution(histCr_m,sizes)
same_pensCr_m = [ v for v in same_pensCr_m. values() ]
diff_pensCr_m = [ v for v in diff_pensCr_m. values() ]

dcr_s=dist(same_pensCr_s,diff_pensCr_s,1465,1456)
dcr_m=dist(same_pensCr_m,diff_pensCr_m,1465,1456)

print(dy_s,dcb_s,dcr_s)
print(dy_m,dcb_m,dcr_m)

/home/mohit/.local/lib/python3.5/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
27.34534890156396 27.572251415849966 21.585624243013772
25.826495834431245 16.506384872947717 22.21749134316207


In [14]:
Y = pd.DataFrame(histY_s)
Cb = pd.DataFrame(histCb_s)
Cr = pd.DataFrame(histCr_s)

all_features = pd.concat([Y,Cb,Cr])
all_features.to_csv('/home/mohit/Desktop/featuresUpdated/OCS-CLBP_S|c_default.csv',sep = ',')

Y = pd.DataFrame(histY_m)
Cb = pd.DataFrame(histCb_m)
Cr = pd.DataFrame(histCr_m)

all_features = pd.concat([Y,Cb,Cr])
all_features.to_csv('/home/mohit/Desktop/featuresUpdated/OCS-CLBP_M|c_default.csv',sep = ',')